In [52]:
import pandas as pd
from typing import List
from app.domains.models import BillVotes, Legislator, LegislatorVotes
from app.utils.csv_paths import *

In [63]:
# read all csv files
bills = pd.read_csv(BILLS_DATA, nrows=100, usecols=["id", "title", "sponsor_id"])
votes = pd.read_csv(VOTES_DATA, nrows=100, usecols=["id", "bill_id"])
vote_results = pd.read_csv(VOTES_RESULTS_DATA, nrows=100, index_col=["id"])

# merge data model
bills = bills.merge(votes, left_on="id", right_on="bill_id")
bills = bills.merge(vote_results, left_on="id_y", right_on="vote_id")
bills

,id_x,title,sponsor_id,id_y,bill_id,legislator_id,vote_id,vote_type
0,2952375,H.R. 5376: Build Back Better Act,412211,3321166,2952375,400440,3321166,2
1,2952375,H.R. 5376: Build Back Better Act,412211,3321166,2952375,17941,3321166,2
2,2952375,H.R. 5376: Build Back Better Act,412211,3321166,2952375,400414,3321166,2
3,2952375,H.R. 5376: Build Back Better Act,412211,3321166,2952375,400380,3321166,2
4,2952375,H.R. 5376: Build Back Better Act,412211,3321166,2952375,412393,3321166,2
5,2952375,H.R. 5376: Build Back Better Act,412211,3321166,2952375,412487,3321166,2
6,2952375,H.R. 5376: Build Back Better Act,412211,3321166,2952375,15367,3321166,2
7,2952375,H.R. 5376: Build Back Better Act,412211,3321166,2952375,412421,3321166,2
8,2952375,H.R. 5376: Build Back Better Act,412211,3321166,2952375,412649,3321166,2
9,2952375,H.R. 5376: Build Back Better Act,412211,3321166,2952375,1269775,3321166,2


In [73]:
bill_votes: List[BillVotes] = []
# bill group by
bill_grouped = bills.groupby(["id_x", "title", "sponsor_id"]).groups

In [74]:
bill_grouped

{(2900994, 'H.R. 3684: Infrastructure Investment and Jobs Act', 400100): [19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37], (2952375, 'H.R. 5376: Build Back Better Act', 412211): [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18]}

In [71]:
for row in bill_grouped.items():
    # supporters
    supporters = bills.query(f"vote_type == 1 & bill_id == {row[0][0]}").value_counts()
    supporters = supporters.groupby("vote_id").sum()

    # opposers
    opposers = bills.query(f"vote_type == 2 & bill_id == {row[0][0]}").value_counts()
    opposers = opposers.groupby("vote_id").sum()

    bill_votes.append(
        BillVotes(
            id=row[0][0],
            bill=row[0][1],
            supporters=supporters.mean(),
            opposors=opposers.mean(),
            primary_sponsor="",
        )
    )

In [72]:
bill_votes

[BillVotes(id=2900994, bill='H.R. 3684: Infrastructure Investment and Jobs Act', supporters=13, opposors=6, primary_sponsor=''),
 BillVotes(id=2952375, bill='H.R. 5376: Build Back Better Act', supporters=6, opposors=13, primary_sponsor='')]